In [ ]:
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
from math import atan2
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/2018/'
#output files
filename_out_nc='F:/data/cruise_data/saildrone/baja-2018/saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1.nc'
#################################################################################
itow_mask1=45 #see code just a bit below for figure to determine where to set flags here
itow_mask2=-110 #see code just a bit below for figure to determine where to set flags here
ISDP = 'Saildrone'
SST_type = 'SSTdepth'
Annex_version = '01.1'
File_version = '01.0'
astr_platform='SD1002'
astr_title = 'Data from Saildrone cruise from SF to Guadalupe Island April-June 2018'
astr_uuid = '92202297-c2d8-4468-9a41-f68386d7bce1'
#input filename
#data_dir = 'F:/data/cruise_data/saildrone/baja-2018/'
dir_in='f:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/2018/'
#dir_out='f:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/'
#read in email
text_file = open('C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/baja/docs/email_for_file.txt', "r")
email_str = text_file.readlines()

In [ ]:
#this goes through each file, concatonates each file and outputs one big file
#the flag == istart_flag is set to zero so that on the first iteration of the loop the array dataset is initialized
istart_flag = 0 
for root, dirs, files in os.walk(dir_in, topdown=False):
    for name in files:
        if name.endswith(".ini"):
            continue
        filename_usv=os.path.join(root, name)
        print(filename_usv)
        dsx = xr.open_dataset(filename_usv)
        dataset2 = xr.open_dataset(filename_usv)
        if istart_flag == 0:
            dataset = dataset2
            istart_flag = 1
            continue
        dataset = xr.concat([dataset, dataset2],'obs')
dataset.to_netcdf(filename_out_nc)


In [ ]:
#add some global attributes

dataset.attrs['title'] = astr_title
dataset.attrs['summary'] = 'none'
dataset.attrs['references'] = 'none'
dataset.attrs['institution'] = 'Saildrone'
dataset.attrs['history'] = 'Saildrone 6-hourly v1 files were used to create this file'
dataset.attrs['comment'] = 'none'
dataset.attrs['license'] = 'free and open'
dataset.attrs['id'] = 'SSTdepth'
dataset.attrs['naming_authority'] = 'org.shipborne-radiometer'
dataset.attrs['product_version'] = '1.0'
dataset.attrs['uuid'] = astr_uuid 
dataset.attrs['l2r_version_id'] = '1.1' 
dataset.attrs['netcdf_version_id'] = '4.6.1'
dataset.attrs['date_created'] = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ") #yyyy-mm-ddThh:mm:ssZ
dataset.attrs['file_quality_level'] = 3
dataset.attrs['spatial_resolution'] = str(distance.mean()*1000)+' m'
dataset.attrs['start_time'] = dates_usv[0].strftime("%Y-%m-%dT%H:%M:%SZ")
dataset.attrs['time_coverage_start'] = dates_usv[0].strftime("%Y-%m-%dT%H:%M:%SZ")
dataset.attrs['stop_time'] = dates_usv[-1].strftime("%Y-%m-%dT%H:%M:%SZ")
dataset.attrs['time_coverage_end'] = dates_usv[-1].strftime("%Y-%m-%dT%H:%M:%SZ")
dataset.attrs['northernmost_latitude'] = lats_usv.max()
dataset.attrs['geospatial_lat_max'] = lats_usv.max()
dataset.attrs['southernmost_latitude'] = lats_usv.min()
dataset.attrs['geospatial_lat_min'] = lats_usv.min()
dataset.attrs['easternmost_longitude'] = lons_usv.max()
dataset.attrs['geospatial_lon_max'] = lons_usv.max()
dataset.attrs['westernmost_longitude'] = lons_usv.min()
dataset.attrs['geospatial_lon_min'] = lons_usv.min()
dataset.attrs['geospatial_lat_units'] = 'degrees_north'
dataset.attrs['geospatial_lat_resolution'] = 'degrees_north'
dataset.attrs['geospatial_lon_units'] = 'degrees_east'
dataset.attrs['geospatial_lon_resolution'] = 'degrees_east'
dataset.attrs['source'] = 'SSTdepth, wind_speed'
dataset.attrs['platform'] = astr_platform
dataset.attrs['sensor'] = str(dataset.TEMP_CTD_MEAN.sensor_description + '_' +\
            dataset.TEMP_CTD_MEAN.model_number + '_' + dataset.TEMP_CTD_MEAN.serial_number + ', ' + \
            dataset.UWND_MEAN.sensor_description + '_' + dataset.UWND_MEAN.model_number + '_' + \
                    dataset.UWND_MEAN.serial_number)
dataset.attrs['metadata_link'] = 'TBD'
dataset.attrs['keywords'] = 'Oceans > Ocean Temperature > Sea Surface Temperature'
dataset.attrs['keywords_vocabulary'] = 'NASA Global Change Master Directory (GCMD) Science Keywords'
#dataset.attrs['standard_name_vocabulary'] = #already there & correct
dataset.attrs['acknowledgment'] = 'The Schmidt Family Foundation, Saildrone, NASA Physical Oceanography'
#dataset.attrs['creator_name'] = #already there & correct
#dataset.attrs['creator_email'] = #already there & correct
#dataset.attrs['creator_url'] = #already there & correct
dataset.attrs['project'] = 'International Shipborne Radiometer Network'
dataset.attrs['publisher_name'] = 'The ISRN Project Office'
dataset.attrs['publisher_url'] = 'http://www.shipborne.radiometer.org'
dataset.attrs['publisher_email'] = email_str
dataset.attrs['processing_level'] = '1.0'
#dataset.attrs['cdm_data_type'] = #already there & correct
#dataset.attrs['featureType'] = #already there & correct

In [ ]:
R = 6373.0 #km
lat1 = np.deg2rad(lats_usv[1:ilen])
lon1 = np.deg2rad(lons_usv[1:ilen])
lat2 = np.deg2rad(lats_usv[0:ilen-1])
lon2 = np.deg2rad(lons_usv[0:ilen-1])
dlon = lon2 - lon1
dlat = lat2 - lat1
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(a**.5, (1 - a)**.5)
distance = R * c



date_1970 = dt.datetime(1970,1,1,0,0,0) # start date is1/1/1970
skin=dataset.TEMP_IR_MEAN[0,:].values
tair=dataset.TEMP_AIR_MEAN[0,:].values
bulk1=dataset.TEMP_CTD_MEAN[0,:].values
bulk2=dataset.TEMP_O2_MEAN[0,:].values
uspd=dataset.UWND_MEAN[0,:].values
vspd=dataset.VWND_MEAN[0,:].values
wing=dataset.HDG_WING[0,:].values
wing_ang=dataset.WING_ANGLE[0,:].values
pitch=dataset.PITCH[0,:].values
heading=dataset.HDG_WING[0,:].values
yaw_heading=dataset.HDG[0,:].values
roll=dataset.ROLL[0,:].values
#sec_since_1970 = dataset.TIME[0,:].values #'seconds since 1970-01-01T00:00:00+00:00'
tdim=len(skin)
print('here')
tem_date=[0]*tdim #np.zeros(tdim)
tem_dy_from=np.zeros(tdim)
tem_lmt=np.zeros(tdim)
#tem_date=[0]*tdim
#tem_dy_from=[0]*tdim
#tem_lmt=[0]*tdim
wdir=np.zeros(tdim)
print(dataset.TIME.shape)
#for i in range(0,10):
#    print(float(dataset.TIME[0,i].values)/86400000000000.)
for i in range(0,tdim):
    tem_dy=float(dataset.TIME[0,i].values)/86400000000000.
    tem_dy_from[i]=float(dataset.TIME[0,i].values)/86400000000000.-float(dataset.TIME[0,0].values)/86400000000000.
    tem_date[i]=date_1970+dt.timedelta(days=tem_dy)  #create new time array that can be queried for year etc
    tem_lmt[i]=tem_date[i].hour+(dataset.LON[0,i].values/360.)*24.
    if tem_lmt[i]>24:
        tem_lmt[i]=tem_lmt[i]-24.
    if tem_lmt[i]<0:
        tem_lmt[i]=tem_lmt[i]+24.
    wdir[i]=atan2(uspd[i],vspd[i])*180/3.14159
    if i<5:
        print(tem_date[i],wdir[i])
        print(tem_dy_from[i])
wspd=(uspd**2+vspd**2)**.5
print('time:', tem_dy_from[1:5],tem_dy_from[-5:-1])


In [ ]:

tem=dataset.HDG_WING[0,:].values
#tem=dataset.HDG[0,:].values
#tem=dataset.SOG[0,:].values
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
print(len(tem_dy_from))
print(len(skin-bulk1))
print(tem_date[-89])
#print(len(tem_dy_from),len(dt))
plt.clf()
plt.scatter(tem_date[:85592],tem[:85592])
plt.xlabel('Day')
plt.ylabel('wdir')
plt.grid()
plt.xlim(datetime(2018,6,10,21),datetime(2018,6,11,1))
#plt.xlim(2018)
#plt.ylim(-2, 2)
fig_fname=adir_tem+"wdir.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)




In [ ]:
dataset2=dataset.sel(obs=slice(0,85592))
mint=dataset2.TIME.min().data
maxt=dataset2.TIME.max().data
dataset2.attrs['time_coverage_start']=str(np.datetime64(mint,'ms'))+'Z'
dataset2.attrs['time_coverage_end']=str(np.datetime64(maxt,'ms'))+'Z'
#dataset
dataset2.to_netcdf(filename_out_nc)
#print('end',dataset.time_coverage_end)
#tem=dataset.TIME.max().data
#print(str(np.datetime64(tem,'ms'))+'Z')
#dataset.attrs['time_coverage_end']=str(np.datetime64(tem,'ms'))+'Z'
#print('end',dataset.time_coverage_end)
#tem.isoformat()


In [ ]:
print(dataset)
print(dataset2)

In [ ]:
#print(dataset)
#print(d)
#tem=dataset.TIME
#print(type(tem)
#dataset2=dataset.where(obs<85592)

print(dataset2)

In [ ]:
dataset=dataset2
import datetime as dt
from datetime import datetime
from math import atan2
date_1970 = dt.datetime(1970,1,1,0,0,0) # start date is1/1/1970
skin=dataset.TEMP_IR_MEAN[0,:].values
tair=dataset.TEMP_AIR_MEAN[0,:].values
bulk1=dataset.TEMP_CTD_MEAN[0,:].values
bulk2=dataset.TEMP_O2_MEAN[0,:].values
uspd=dataset.UWND_MEAN[0,:].values
vspd=dataset.VWND_MEAN[0,:].values
wing=dataset.HDG_WING[0,:].values
wing_ang=dataset.WING_ANGLE[0,:].values
pitch=dataset.PITCH[0,:].values
heading=dataset.HDG_WING[0,:].values
yaw_heading=dataset.HDG[0,:].values
roll=dataset.ROLL[0,:].values
#sec_since_1970 = dataset.TIME[0,:].values #'seconds since 1970-01-01T00:00:00+00:00'
tdim=len(skin)
print('here')
tem_date=[0]*tdim #np.zeros(tdim)
tem_dy_from=np.zeros(tdim)
tem_lmt=np.zeros(tdim)
#tem_date=[0]*tdim
#tem_dy_from=[0]*tdim
#tem_lmt=[0]*tdim
wdir=np.zeros(tdim)
print(dataset.TIME.shape)
#for i in range(0,10):
#    print(float(dataset.TIME[0,i].values)/86400000000000.)
for i in range(0,tdim):
    tem_dy=float(dataset.TIME[0,i].values)/86400000000000.
    tem_dy_from[i]=float(dataset.TIME[0,i].values)/86400000000000.-float(dataset.TIME[0,0].values)/86400000000000.
    tem_date[i]=date_1970+dt.timedelta(days=tem_dy)  #create new time array that can be queried for year etc
    tem_lmt[i]=tem_date[i].hour+(dataset.LON[0,i].values/360.)*24.
    if tem_lmt[i]>24:
        tem_lmt[i]=tem_lmt[i]-24.
    if tem_lmt[i]<0:
        tem_lmt[i]=tem_lmt[i]+24.
    wdir[i]=atan2(uspd[i],vspd[i])*180/3.14159
    if i<5:
        print(tem_date[i],wdir[i])
        print(tem_dy_from[i])
wspd=(uspd**2+vspd**2)**.5
print('time:', tem_dy_from[1:5],tem_dy_from[-5:-1])


In [ ]:
#filename_nc='C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/data_so_far.nc'
#dataset = xr.open_dataset(filename_nc)

#lats=dataset['LAT']
#lons=dataset['LON']
ilen=(len(dataset.LAT['obs']))
print(dataset.LAT[0,1].values*1)
print(type(dataset))
lats=dataset.LAT[0,:].values
lons=dataset.LON[0,:].values
print(lats.shape)
ilen=len(lats)
print(ilen)

In [ ]:
import csv
import simplekml

#inputfile = csv.reader(open('foo.csv','r'))
kml=simplekml.Kml()
ls = kml.newlinestring(name="Saildrone path")
ilen=len(lats)
#inputfile.next()
#for row in inputfile:
for i in range(0,ilen): 
    #print([(lons[i],lats[i])])
    ls.coords.addcoordinates([(lons[i],lats[i])]) #<-- IMPORTANT! Longitude first, Latitude second.
kml.save('F:/data/cruise_data/saildrone/baja-2018/Saildrone_line.kml');

In [ ]:
fig, ax = plt.subplots()
ax.plot(lons,lats)
print(min(lons),max(lons))
print(min(lats),max(lats))
#print(ls.coords)

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
#file='f:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/2018/06\09\saildrone-gen_4-baja_2018-EP-sd1002-20180609T0600-20180609T1159-1_min-v1.1528549203.nc'


In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(11.7,8.3))
            #Custom adjust of the subplots
plt.subplots_adjust(left=0.05,right=0.95,top=0.90,bottom=0.05,wspace=0.15,hspace=0.05)
ax = plt.subplot(111)
            #Let's create a basemap of the world
m = Basemap(projection='merc', lat_0 = 27, lon_0 = -120,    resolution = 'l', area_thresh = 0.1,    
                          llcrnrlon=-126.0, llcrnrlat=25.0,urcrnrlon=-114.0, urcrnrlat=38.)
m.fillcontinents(color='coral',lake_color='white')
x,y = m(lons,lats)
m.plot(x, y, 'bo-', markersize=1, linewidth = 1)
        #plt.show()
fig_fname="f:/data/cruise_data/saildrone/baja-2018/figs/track.png"
fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)
#plt.savefig(fig_fname,dpi=300)
print(fig_fname)
#fig, ax = plt.subplots()
#ax.plot(lons,lats)
#plt.show()

In [ ]:
#wing=dataset.HDG_WING[0,:].value
#print(min(wing),max(wing))
#print(wing[-300:-1])
#print(type(skin))
#print(type(tem_dy_from))

In [ ]:
adir_tem='f:/data/cruise_data/saildrone/baja-2018/figs/'
#adir_tem='C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/'
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
fig, (ax1,ax2) = plt.subplots(1,2) #figure(2,figsize=(11.7,8.3))
ax1.scatter(wspd, bulk1-bulk2,c=tem_lmt,s=2) #,label=tem_lmt) #, s=area, c=colors, alpha=0.5)
ax1.set(xlabel='Wind Speed (ms$^{-1}$)', ylabel='CTD minus O2 Bulk SST (K)')
#fig.colorbar(mappable='none') #,location="bottom",label='wind speed')
ax2.scatter(tem_lmt, bulk1-bulk2,c=wspd,s=2) #,label=tem_lmt) #, s=area, c=colors, alpha=0.5)
ax2.set(xlabel='Local Time (hrs)')
fig_fname=adir_tem+"wndbulk.png"
fig.savefig(fig_fname, transparent=False, format='png')
plt.show()
print(fig_fname)


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2) #figure(2,figsize=(11.7,8.3))
ax1.scatter(wspd, skin-bulk1,c=tem_lmt,s=2) #,label=tem_lmt) #, s=area, c=colors, alpha=0.5)
ax1.set(xlabel='Wind Speed (ms$^{-1}$)', ylabel='Skin minus CTD bulk SST (K)')
ax2.scatter(tem_lmt, skin-bulk1,c=wspd,s=2) #,label=tem_lmt) #, s=area, c=colors, alpha=0.5)
ax2.set(xlabel='Local Time (hrs)')
fig_fname=adir_tem+"wndbulkdiff.png"
fig.savefig(fig_fname, transparent=False, format='png')
plt.show()
print(fig_fname)

In [ ]:
from matplotlib import pyplot as plt 
import numpy as np  
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
num_bins = 100
dt=skin-bulk1
hist, bins =np.histogram(dt,bins=num_bins,range=(-1,1))
#hist, bins = np.histogram(x, bins=50)
width = 1 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width,alpha=0.5)
ax1.set(xlabel='Skin - Bulk (K))')
plt.show()#print(dt.min,dt.max)
fig_fname="f:/data/cruise_data/saildrone/baja-2018/figs/skin_hist.png"
fig.savefig(fig_fname, transparent=False, format='png')
#plt.show()
print(fig_fname)

In [ ]:
from matplotlib import pyplot as plt 
import numpy as np  
from numpy import array
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
num_bins = 100
dt=skin-bulk1
daynight=np.zeros(len(tem_lmt))
tem_lmt2=array(tem_lmt)
print(type(tem_lmt),type(tem_lmt2))
daynight[tem_lmt2<6]=1
daynight[tem_lmt2>18]=1
dtnight=dt[np.where(daynight==1)]
dtday=dt[np.where(daynight==0)]
hist, bins =np.histogram(dtday,bins=num_bins,range=(-1,1))
hist2, bins2 =np.histogram(dtnight, bins=num_bins,range=(-1,1))
#hist, bins = np.histogram(x, bins=50)
width = 1 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
p1=plt.bar(center, hist, align='center', width=width,alpha=0.5,color='r')
width2 = 1 * (bins2[1] - bins2[0])
center2 = (bins2[:-1] + bins2[1:]) / 2
p2=plt.bar(center2, hist2, align='center', width=width2,alpha=0.5,color='b')
plt.xlabel('Skin - Bulk (K))',fontsize=18)
plt.ylabel('Num. Obs',fontsize=18)
plt.legend((p1[0], p2[0]), ('Day', 'Night'))
#plt.show()#print(dt.min,dt.max)
fig_fname=adir_tem+"skin_hist.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)

In [ ]:
print(min(lats),max(lats))
print(min(lons),max(lons))
print(tem_date[1],tem_date[-1])

fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
num_bins = 100
dt=skin-bulk1

print(min(wdir),max(wdir))
print(min(wing),max(wdir))
plt.hist(wing,bins=360)
plt.xlabel('Wing heading; 0 is straight back along USV)',fontsize=18)
plt.ylabel('Num. Obs',fontsize=18)
#plt.legend((p1[0], p2[0]), ('Day', 'Night'))
#plt.show()#print(dt.min,dt.max)
fig_fname="f:/data/cruise_data/saildrone/baja-2018/figs/hist_wing.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)

In [ ]:
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
plt.scatter(wing,dt,c=wspd)
plt.xlabel('Wing Direction (deg)')
plt.ylabel('Skin minus T$_{depth}$')
fig_fname=adir_tem+"skinbulk_wing.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)


In [ ]:
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
plt.scatter(wdir,dt,c=wspd)
plt.xlabel('Wind Direction (deg)')
plt.ylabel('Skin minus T$_{depth}$')
fig_fname=adir_tem+"skinbulk_wind.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)

In [ ]:
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
print(len(tem_dy_from))
print(len(skin-bulk1))
#print(len(tem_dy_from),len(dt))
plt.scatter(tem_dy_from,skin-bulk1,c=wspd)
plt.xlabel('Mission Day')
plt.ylabel('Skin minus T$_{depth}$')
#plt.xlim(2018)
#plt.ylim(-2, 2)
fig_fname=adir_tem+"skinbulk_TS.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)

In [ ]:
print(len(tem_dy_from))
print(len(skin-bulk1),tdim)
item=np.zeros(tdim)
for i in range(0,tdim):
    item[i]=int(round(tem_dy_from[i]/5.))
print(item[0:5000:500],max(item),item[tdim-1])
print(tem_dy_from[0:5000:500],tem_dy_from[tdim-1])
dt=skin-bulk1
i=0
mindy=min(tem_date)
maxdy=max(tem_date)
print(maxdy-mindy)
intdy=(maxdy-mindy)/12.
print(len(tem_dy_from[item==i]))
fig, ax1 = plt.subplots() #figure(2,figsize=(11.7,8.3))
#plt.scatter(tem_dy_from[item==i],dt[item==i],s=1,c=pitch[item==i])
plt.scatter(tem_date,dt,s=1,c=pitch)
   # plt.scatter(tem_dy_from[item==i],dt[item==i]-.5,s=1,c=roll[item==i])
   # plt.scatter(tem_dy_from[item==i],dt[item==i]-1.0,s=1,c=heading[item==i])
   # plt.scatter(tem_dy_from[item==i],dt[item==i]-1.5,s=1,c=wing_ang[item==i])
plt.xlabel('Date')
plt.ylabel('T$_{IR}$ - T$_{CTD}$')
for i in range(0,12):
    plt.ylim(-1, 1.6)
    plt.xlim(mindy+i*intdy,mindy+(i+1)*intdy)
    #plt.show()
    fig_fname=adir_tem+'skin-bulk/skinbulk_TS' + str(i) +'.png'
    print(fig_fname)
    fig.savefig(fig_fname, transparent=False, format='png')


In [ ]:
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)

cmap=palette
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

ytem=skin-bulk1
xtem=tair-bulk1
xtem[np.isnan(ytem)]=np.nan
ytem[np.isnan(xtem)]=np.nan
x = xtem[~np.isnan(xtem)]
y = ytem[~np.isnan(ytem)]
H, xedges, yedges = np.histogram2d(x,y, bins=100,normed='True')
X, Y = np.meshgrid(xedges, yedges)
X[X==0] = np.nan
Y[Y==0] = np.nan
ax1.pcolormesh(X, Y, H,cmap=cmap,vmin=0.01,vmax=1)
ax1.set_aspect('equal')
cmap = plt.get_cmap('jet')
cmap.set_under(color='white')
plt.ylim(-1,2)
plt.xlim(-4, 0)
ax1.set_xlabel('$\Delta$T$_{skin-bulk}$ (K)')
ax1.set_ylabel('$\Delta$T$_{air-bulk}$ (K)')

ytem=skin-bulk1
xtem=wspd
xtem[np.isnan(ytem)]=np.nan
ytem[np.isnan(xtem)]=np.nan
x = xtem[~np.isnan(xtem)]
y = ytem[~np.isnan(ytem)]
H, xedges, yedges = np.histogram2d(x,y, bins=100,normed='True')
X, Y = np.meshgrid(xedges, yedges)
X[X==0] = np.nan
Y[Y==0] = np.nan
ax2.pcolormesh(X, Y, H,cmap=cmap,vmin=0.01,vmax=1)
ax2.set_aspect('equal')
cmap = plt.get_cmap('jet')
cmap.set_under(color='white')
plt.ylim(-1,2)
plt.xlim(0,10)
ax2.set_ylabel('$\Delta$T$_{skin-bulk}$ (K)')
ax2.set_xlabel('U (ms$^{-1}$)')

ytem=skin-bulk1
xtem=wing_ang
xtem[np.isnan(ytem)]=np.nan
ytem[np.isnan(xtem)]=np.nan
x = xtem[~np.isnan(xtem)]
y = ytem[~np.isnan(ytem)]
H, xedges, yedges = np.histogram2d(x,y, bins=100,normed='True')
X, Y = np.meshgrid(xedges, yedges)
X[X==0] = np.nan
Y[Y==0] = np.nan
ax3.pcolormesh(X, Y, H,cmap=cmap,vmin=0.01,vmax=1)
ax3.set_aspect('equal')
cmap = plt.get_cmap('jet')
cmap.set_under(color='white')
plt.ylim(-1,2)
plt.xlim(0,360)
ax3.set_ylabel('$\Delta$T$_{skin-bulk}$ (K)')
ax3.set_xlabel('wing angle ($\circ$)')

ytem=skin-bulk1
xtem=roll
xtem[np.isnan(ytem)]=np.nan
ytem[np.isnan(xtem)]=np.nan
x = xtem[~np.isnan(xtem)]
y = ytem[~np.isnan(ytem)]
H, xedges, yedges = np.histogram2d(x,y, bins=100,normed='True')
X, Y = np.meshgrid(xedges, yedges)
X[X==0] = np.nan
Y[Y==0] = np.nan
ax4.pcolormesh(X, Y, H,cmap=cmap,vmin=0.01,vmax=1)
ax4.set_aspect('equal')
cmap = plt.get_cmap('jet')
cmap.set_under(color='white')
plt.ylim(-1,2)
plt.xlim(0,360)
ax4.set_ylabel('$\Delta$T$_{skin-bulk}$ (K)')
ax4.set_xlabel('wing angle ($\circ$)')


fig_fname=adir_tem+"hist.png"
fig.savefig(fig_fname, transparent=False, format='png')
print(fig_fname)


In [ ]:
filename_goes_sst = 'F:/data/cruise_data/saildrone/baja-2018/goes_sst_collocation_data.npy'
gsst=np.load(filename_goes_sst)

In [ ]:
dbulk=bulk1-bulk2
dbulk=dbulk[~np.isnan(dbulk)]
print(dbulk[0:10])
print(dbulk.mean())
print(dbulk.std())